<a href="https://colab.research.google.com/github/WanjohiChristopher/Zindi-Kenya-Sign-Language-Classification-Hack/blob/main/Kenyan_Sign_LanguageClassification_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q  fastai==1.0.61

In [ ]:
# check version of fastai
import fastai

fastai.__version__

In [ ]:
# !unzip '/content/drive/MyDrive/zindi taskmate/Images.zip' -d '/content/drive/MyDrive/zindi taskmate/Images'

In [ ]:
import numpy as np 
import pandas as pd 

from fastai import *
from fastai.vision import *
from pathlib import Path
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
image_path = '../input/taskmate/data/Images'

In [ ]:
import os
data = os.listdir(image_path)

In [ ]:
len(data)

### tfms helps to do data augmentation on the data to have a better pixel values

In [ ]:
tfms=get_transforms(do_flip=True,flip_vert=True,
                    max_lighting=0.5,max_zoom=2.0,
                    max_warp=0.2,max_rotate=45)

### Read the data and pass the data augmentation function

In [ ]:
import pandas as pd
df = pd.read_csv("../input/taskmate/data/Train.csv")
df1 = pd.read_csv("../input/taskmate/data/Test.csv")

display(df.shape[0] + df1.shape[0])

In [ ]:
df.head()

In [ ]:
list_of_images = [images.split(".")[0] for images in os.listdir("../input/taskmate/data/Images")]

In [ ]:
df.shape

In [ ]:
df

In [ ]:
data = ImageDataBunch.from_df(image_path,
                              df,
                              size=224,
                              valid_pct=0.20,
                              ds_tfms=tfms,
                              num_workers=5,
                              bs=10,
                             suffix='.jpg').normalize(imagenet_stats)

In [ ]:
# There are 9 classes!
print(data.c) 

In [ ]:
# # Look at some examples
# data.show_batch(rows=1, figsize=(5, 5)) 

### Modelling
Using Efficientnet Model

In [ ]:
!pip install efficientnet-pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[error_rate, accuracy])
model = EfficientNet.from_name('efficientnet-b0')
#model._fc = nn.Linear(1280, data.c)
#learn = Learner(data, model,metrics=[error_rate, accuracy])
#learn.fit(n_epoch=200, lr=1e-7, cbs=EarlyStoppingCallback(monitor='valid_loss', min_delta=0.1, patience=2))



In [ ]:
from fastai.callback import *

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=data.c)

learn = Learner(data, model, metrics=[error_rate, accuracy])

learn.fit_one_cycle(65,1e-4)

In [ ]:
# # Fit
# num_epochs = 25
# fastai.vision.callbacks.EarlyStoppingCallback(learn, monitor='accuracy', min_delta=0.01, patience=3),
# learn.fit_one_cycle(num_epochs,1e-3)

## predict on test set

In [ ]:
#https://towardsdatascience.com/deep-learning-image-classification-with-fast-ai-fc4dc9052106

In [ ]:
test_df = pd.read_csv("../input/taskmate/data/Test.csv")

In [ ]:
test_df.head()

In [ ]:
test = ImageList.from_df(test_df, image_path, suffix='.jpg')

In [ ]:
data.add_test(test)

In [ ]:
#predictions

In [ ]:
preds_test,y_test = learn.get_preds(DatasetType.Test)

In [ ]:
preds_test.shape

In [ ]:
sub=pd.read_csv('../input/taskmate/data/SampleSubmission.csv')
k=[]
for col in sub.columns: 
    k.append(col) # creating list of the label

In [ ]:
k

In [ ]:
import os
submission = pd.DataFrame()
submission["ID"] = test_df["img_IDS"]
for i, c in enumerate(learn.data.classes):
    print(c)
    submission[c] = preds_test[:,i]
submission.head()

In [ ]:
submission.to_csv('./subf_resnet_65tta.csv', index=False)